- 대회 데이터 셋 오류로 인한 데이터 전처리
  - https://dacon.io/competitions/official/235745/talkboard/403708?page=1&dtype=recent

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

train.shape, test.shape

((2952, 15), (1022, 14))

In [3]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
       '도보 10분거리 내 버스정류장 수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [4]:
train.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수', '등록차량수']

test.columns = ['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수',
       '10분내버스정류장수', '단지내주차면수']

### 데이터 오류로 인한 데이터 제외
- 테스트셋에서 평가 제외되는 데이터는 'C2675'(2번 사항에 해당), 'C2335', 'C1327'(3번 사항에 해당) 3개 단지입니다.

In [5]:
train.isna().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         569
임대료           569
10분내지하철수      211
10분내버스정류장수      4
단지내주차면수         0
등록차량수           0
dtype: int64

In [6]:
train.단지코드.unique()

array(['C2483', 'C2515', 'C1407', 'C1945', 'C1470', 'C1898', 'C1244',
       'C1171', 'C2073', 'C2513', 'C1936', 'C2049', 'C2202', 'C1925',
       'C2576', 'C1312', 'C1874', 'C2650', 'C2416', 'C2013', 'C1424',
       'C2100', 'C2621', 'C2520', 'C2319', 'C1616', 'C1704', 'C2258',
       'C1032', 'C2038', 'C1859', 'C1722', 'C1850', 'C2190', 'C1476',
       'C1077', 'C1068', 'C1983', 'C2135', 'C2034', 'C1109', 'C1497',
       'C2289', 'C2597', 'C2310', 'C1672', 'C2132', 'C1439', 'C1613',
       'C2216', 'C1899', 'C1056', 'C2644', 'C1206', 'C2481', 'C1718',
       'C1655', 'C1430', 'C1775', 'C1519', 'C2221', 'C1790', 'C2109',
       'C1698', 'C1866', 'C1005', 'C1004', 'C1875', 'C2156', 'C2212',
       'C2401', 'C2571', 'C1175', 'C1833', 'C2445', 'C1885', 'C2368',
       'C2016', 'C2371', 'C2536', 'C2538', 'C1014', 'C1592', 'C1867',
       'C2326', 'C1015', 'C1620', 'C1049', 'C2000', 'C2097', 'C1668',
       'C1689', 'C1234', 'C2514', 'C1368', 'C1057', 'C2336', 'C1026',
       'C2256', 'C19

In [7]:
# 우선 train 데이터 셋 확인
train.loc[ ((train['단지코드']=='C2675') | 
           (train['단지코드']=='C2335') |
           (train['단지코드']=='C1327') ) , :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수


In [8]:
# 테스트 데이터 셋 확인
test.loc[ ((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :].head(3)

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
579,C2675,512,아파트,경기도,국민임대,36.65,130,9.0,A,18476000,154790,0.0,3.0,1016.0
580,C2675,512,아파트,경기도,국민임대,46.90,44,9.0,A,34082000,232200,0.0,3.0,1016.0
581,C2675,512,아파트,경기도,국민임대,46.90,80,9.0,A,34082000,232200,0.0,3.0,1016.0


In [9]:
# 테스트 데이터 셋에서 세개의 코드 데이터를 없애기
test = test.loc[ ~((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :]
test.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C1072,754,아파트,경기도,국민임대,39.79,116,14.0,H,22830000,189840,0.0,2.0,683.0
1,C1072,754,아파트,경기도,국민임대,46.81,30,14.0,A,36048000,249930,0.0,2.0,683.0
2,C1072,754,아파트,경기도,국민임대,46.90,112,14.0,H,36048000,249930,0.0,2.0,683.0
3,C1072,754,아파트,경기도,국민임대,46.90,120,14.0,H,36048000,249930,0.0,2.0,683.0
4,C1072,754,아파트,경기도,국민임대,51.46,60,14.0,H,43497000,296780,0.0,2.0,683.0


In [10]:
# 확인
test.loc[ ((test['단지코드']=='C2675') | 
           (test['단지코드']=='C2335') |
           (test['단지코드']=='C1327') ) , :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수


### 오류 데이터 처리
- ※ 동일한 단지에 코드가 2개로 부여된 단지 코드 (3쌍) : ['C2085', 'C1397'], ['C2431', 'C1649'], ['C1036', 'C2675']
    - (참고 사항) 주차면수는 하나의 단지임을 전제로 산정된 것이고 총세대수는 두 개 단지의 합계입니다. 다만 등록차량대수는 ['C2085', 'C1397'] 단지의 경우 동일 수치

In [11]:
train.loc[ train['단지코드']=='C2085',  "총세대수" ] = 1339
train.loc[ train['단지코드']=='C1397',  "총세대수" ] = 1339

In [12]:
# 단지코드를 C2085,C1397 => N2085로 변경
print( train.loc[ train['단지코드']=='C2085', : ].shape  )
print( train.loc[ train['단지코드']=='C1397', : ].shape  )

(8, 15)
(6, 15)


In [13]:
# 변경 후, 처리 후, 단지코드를 N을 붙여 N2085로 변경
train.loc[ train['단지코드']=='C2085',  "단지코드" ] = 'N2085'
train.loc[ train['단지코드']=='C1397',  "단지코드" ] = 'N2085'

In [14]:
train.loc[ train['단지코드']=='N2085', : ].shape

(14, 15)

### 오류 코드 변경
- C2431, C1649의 총세대수를 1047로 변경
- C2431, C1649의 등록차량대수를 1214로 변경
- C2431, C1649의 단지코드를 N2431로 변경

In [15]:
a = train.loc[ train['단지코드']=='C2431', : ]
b = train.loc[ train['단지코드']=='C1649', : ]

print(  a.shape, b.shape )
print( a['총세대수'], b['총세대수'])
print( a['등록차량수'], b['등록차량수'])

(2, 15) (4, 15)
2372    472
2373    472
Name: 총세대수, dtype: int64 2315    575
2316    575
2317    575
2318    575
Name: 총세대수, dtype: int64
2372    359.0
2373    359.0
Name: 등록차량수, dtype: float64 2315    855.0
2316    855.0
2317    855.0
2318    855.0
Name: 등록차량수, dtype: float64


In [16]:
train.loc[ train['단지코드']=='C2431',  "총세대수" ] = 1047
train.loc[ train['단지코드']=='C1649',  "총세대수" ] = 1047

train.loc[ train['단지코드']=='C2431',  "등록차량수" ] = 1214
train.loc[ train['단지코드']=='C1649',  "등록차량수" ] = 1214

train.loc[ train['단지코드']=='C2431',  "단지코드" ] = 'N2431'
train.loc[ train['단지코드']=='C1649',  "단지코드" ] = 'N2431'

In [17]:
train.loc[ train['단지코드']=='N2431', : ].shape

(6, 15)

### 오류 코드 변경
- C1036의 총세대수를 1243로 변경
- C1036의 단지코드를 N1036로 변경

In [18]:
a = train.loc[ train['단지코드']=='C2675', : ]
b = train.loc[ train['단지코드']=='C1036', : ]
a.shape, b.shape

((0, 15), (7, 15))

In [19]:
train.loc[ train['단지코드']=='C1036',  "총세대수" ] = 1243
train.loc[ train['단지코드']=='C1036',  "단지코드" ] = 'N1036'

In [20]:
train.loc[ train['단지코드']=='N1036', : ].shape

(7, 15)

### 오류 3
3. 단지코드 등 기입 실수로 데이터 정제 과정에서 매칭 오류 발생  
 - (오류 내용) 단지코드 등 기입 실수로 총세대수가 주차면수에 비해 과하게 많거나 적은 경우가 발생하였고, 점검 결과 일부 데이터의 단지코드, 총세대수, 주차면수 등에서 오류가 검출되었습니다.
 - (발생 원인) 원천데이터 수집 과정에서 단지 코드 등이 잘못 기입되었고 이를 인지하지 못한 채 데이터 정제를 하여 오류가 발생하였습니다.
 - (관련 데이터) 아래와 같이 총 9개 단지에서 같은 문제가 확인되었습니다. 
※ 실수가 발생한 단지 코드 (9개 단지) : ['C2335', 'C1327', 'C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
 - C2335, C1327 단지는 테스트셋, 나머지는 트레인셋 입니다.

### 오류 처리
- train 데이터 셋에 오류 발생 코드를 ERR04로 변경 후, 데이터 셋을 두개로 분리

In [21]:
train.loc[ train['단지코드']=='C1095',  "단지코드" ] = 'ERR04_1095'
train.loc[ train['단지코드']=='C2051',  "단지코드" ] = 'ERR04_2051'
train.loc[ train['단지코드']=='C1218',  "단지코드" ] = 'ERR04_1218'
train.loc[ train['단지코드']=='C1894',  "단지코드" ] = 'ERR04_1894'
train.loc[ train['단지코드']=='C2483',  "단지코드" ] = 'ERR04_2483'
train.loc[ train['단지코드']=='C1502',  "단지코드" ] = 'ERR04_1502'
train.loc[ train['단지코드']=='C1988',  "단지코드" ] = 'ERR04_1988'

In [22]:
train.loc[ train['단지코드'].str.contains('ERR'), :].shape

(56, 15)

In [23]:
train.loc[ train['단지코드'].str.contains('ERR'), :]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,등록차량수
0,ERR04_2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,ERR04_2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,ERR04_2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,ERR04_2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,ERR04_2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
5,ERR04_2483,900,아파트,경상북도,국민임대,51.95,89,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
6,ERR04_2483,900,아파트,경상북도,국민임대,51.95,135,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
7,ERR04_2483,900,아파트,경상북도,국민임대,59.88,86,38.0,A,30357000,214270,0.0,3.0,1425.0,1015.0
1228,ERR04_1988,475,아파트,전라남도,국민임대,36.63,200,12.0,A,12026000,87940,0.0,2.0,722.0,402.0
1229,ERR04_1988,475,아파트,전라남도,국민임대,36.63,43,12.0,A,12026000,87940,0.0,2.0,722.0,402.0


In [24]:
train_df = train.copy()
train_df_errno = train.loc[ ~train['단지코드'].str.contains('ERR'), :]
test_df = test.copy()

In [25]:
train_df_errno.reset_index(inplace=True)

In [26]:
train_df.shape, train_df_errno.shape, test_df.shape

((2952, 15), (2896, 16), (1008, 14))

### 오류 1 전용면적별 세대의 합계와 총세대수가 일치하지 않는 오류
- 차이가 14세대 이하인 48개 단지 - ['C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 'C1584', 'C2298', 'C2225', 'C1218', 'C1970', C1732', 'C2433', 'C1894', 'C1156', 'C2142', 'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189']
- 차이가 94~452세대인 10개 단지(크기순) - ['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804’]

In [27]:
# 데이터 병합
all_df = pd.concat([train_df_errno, test_df], join='inner', ignore_index=True)
all_df

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0
5,C2515,545,아파트,경상남도,국민임대,51.97,106,17.0,A,23042000,190090,0.0,3.0,624.0
6,C2515,545,아파트,경상남도,국민임대,51.97,26,17.0,A,23042000,190090,0.0,3.0,624.0
7,C1407,1216,아파트,대전광역시,국민임대,30.95,288,13.0,A,15620000,127350,1.0,1.0,1285.0
8,C1407,1216,아파트,대전광역시,국민임대,30.99,68,13.0,A,15620000,127350,1.0,1.0,1285.0
9,C1407,1216,아파트,대전광역시,국민임대,30.99,34,13.0,A,15620000,127350,1.0,1.0,1285.0


In [28]:
# 오류1 14세대 이하 처리
group1 = ['C1925', 'C1312', 'C2013', 'C1424', 'C2520', 'C2319', 'C1850', 'C1068', 'C2644', 'C2156', 
          'C2453', 'C1910', 'C2139', 'C2508', 'C1695', 'C2556', 'C2362', 'C2568', 'C2245', 'C2549', 
          'C1584', 'C2298', 'C2225', 'C1218', 'C1970', 'C1732', 'C2433', 'C1894', 'C1156', 'C2142', 
          'C2186', 'C2411', 'C1812', 'C1030', 'C1749', 'C1349', 'C2043', 'C1229', 'C2363', 'C1414', 
          'C2174', 'C2404', 'C1683', 'C1038', 'C2456', 'C1266', 'C1267', 'C2189' ]

for one in group1:
    all_df.loc[ all_df['단지코드'] == one, "단지코드_Type" ] = 1

In [29]:
# 오류1 94~452세대 10개 단지 처리
group2 = ['C1490', 'C2497', 'C2620', 'C1344', 'C1024', 'C2470', 'C1206', 'C1740', 'C2405', 'C1804']

for one in group2:
    all_df.loc[ all_df['단지코드'] == one, "단지코드_Type" ] = 2

In [30]:
# 차이가 94~452세대인 10개단지 처리하기
all_df.loc[  all_df['단지코드_Type'].isna(), "단지코드_Type"] = 3
all_df['단지코드_Type'].unique()

array([3., 1., 2.])

### 결측치 처리

In [31]:
all_df.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         749
임대료           749
10분내지하철수      249
10분내버스정류장수      4
단지내주차면수         0
단지코드_Type       0
dtype: int64

In [32]:
all_df.loc[all_df['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,단지코드_Type
3092,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0,1.0
3154,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0,3.0


In [33]:
all_df.loc[ 3092, "자격유형"] = 'A'
all_df.loc[ 3154, "자격유형"] = 'C'

In [34]:
all_df.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         749
임대료           749
10분내지하철수      249
10분내버스정류장수      4
단지내주차면수         0
단지코드_Type       0
dtype: int64

In [35]:
# 10분내 버스 정류장수 결측치 처리
grouped = all_df.groupby(['임대건물구분', '지역'])
group1 = grouped.get_group( ('아파트', '경상남도')  )
group1['10분내버스정류장수'].mean()

3.9830028328611897

In [36]:
val = group1['10분내버스정류장수'].mean()
all_df.loc[ all_df['10분내버스정류장수'].isnull(), "10분내버스정류장수"] = val
all_df.loc[all_df['10분내버스정류장수'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,단지코드_Type


In [37]:
# 라벨 인코딩 - 문자를 숫자로 변경
gubun1 = {'아파트':1, '상가':2}

gubun2 = {'경상남도':1, '대전광역시':2, '경기도':3, '전라북도':4, 
          '강원도':5, '광주광역시':6, '충청남도':7, '부산광역시':8, 
          '제주특별자치도':9, '울산광역시':10, '충청북도':11, '전라남도':12, 
          '경상북도':13, '대구광역시':14, '서울특별시':15, '세종특별자치시':16}

gubun3 = {'국민임대':1, '공공임대(50년)':2, '영구임대':3, '임대상가':4, 
          '공공임대(10년)':5, '공공임대(분납)':6, '장기전세':7, '공공분양':8, 
          '행복주택':9, '공공임대(5년)':10}

gubun4 = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5, 
           'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 
           'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }

all_df['임대건물구분_lbl'] = all_df['임대건물구분'].map(gubun1)
all_df['지역_lbl'] = all_df['지역'].map(gubun2)
all_df['공급유형_lbl'] = all_df['공급유형'].map(gubun3)
all_df['자격유형_lbl'] =all_df['자격유형'].map(gubun4).astype(int)

all_df

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,단지코드_Type,임대건물구분_lbl,지역_lbl,공급유형_lbl,자격유형_lbl
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0,3.0,1,1,1,1
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0,3.0,1,1,1,1
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0,3.0,1,1,1,1
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0,3.0,1,1,1,1
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0,3.0,1,1,1,1
5,C2515,545,아파트,경상남도,국민임대,51.97,106,17.0,A,23042000,190090,0.0,3.0,624.0,3.0,1,1,1,1
6,C2515,545,아파트,경상남도,국민임대,51.97,26,17.0,A,23042000,190090,0.0,3.0,624.0,3.0,1,1,1,1
7,C1407,1216,아파트,대전광역시,국민임대,30.95,288,13.0,A,15620000,127350,1.0,1.0,1285.0,3.0,1,2,1,1
8,C1407,1216,아파트,대전광역시,국민임대,30.99,68,13.0,A,15620000,127350,1.0,1.0,1285.0,3.0,1,2,1,1
9,C1407,1216,아파트,대전광역시,국민임대,30.99,34,13.0,A,15620000,127350,1.0,1.0,1285.0,3.0,1,2,1,1


In [38]:
# 구간분할 해주기(총세대수에 대해서)
all_df['단지코드'] = all_df['단지코드'].astype("category")
all_df['단지코드_lbl'] = all_df['단지코드'].cat.codes

all_df['qcut_총세대수'] = pd.qcut(all_df['총세대수'], 5, labels=False)
all_df['qcut_총세대수'].unique()

array([1, 3, 2, 4, 0], dtype=int64)

In [39]:
train_df_errno.shape, test_df.shape

((2896, 16), (1008, 14))

In [40]:
train_df_last = all_df.iloc[0:2896,:]
test_df_last = all_df.iloc[2896:,:]

train_df_last.shape, test_df_last.shape

((2896, 21), (1008, 21))

In [41]:
train_df_last = pd.concat([train_df_last, train_df_errno['등록차량수'] ], axis=1)
train_df_last

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,...,10분내버스정류장수,단지내주차면수,단지코드_Type,임대건물구분_lbl,지역_lbl,공급유형_lbl,자격유형_lbl,단지코드_lbl,qcut_총세대수,등록차량수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,...,3.0,624.0,3.0,1,1,1,1,492,1,205.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,...,3.0,624.0,3.0,1,1,1,1,492,1,205.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,...,3.0,624.0,3.0,1,1,1,1,492,1,205.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,...,3.0,624.0,3.0,1,1,1,1,492,1,205.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,...,3.0,624.0,3.0,1,1,1,1,492,1,205.0
5,C2515,545,아파트,경상남도,국민임대,51.97,106,17.0,A,23042000,...,3.0,624.0,3.0,1,1,1,1,492,1,205.0
6,C2515,545,아파트,경상남도,국민임대,51.97,26,17.0,A,23042000,...,3.0,624.0,3.0,1,1,1,1,492,1,205.0
7,C1407,1216,아파트,대전광역시,국민임대,30.95,288,13.0,A,15620000,...,1.0,1285.0,3.0,1,2,1,1,131,3,1064.0
8,C1407,1216,아파트,대전광역시,국민임대,30.99,68,13.0,A,15620000,...,1.0,1285.0,3.0,1,2,1,1,131,3,1064.0
9,C1407,1216,아파트,대전광역시,국민임대,30.99,34,13.0,A,15620000,...,1.0,1285.0,3.0,1,2,1,1,131,3,1064.0


In [42]:
# 새로운 특징만들고 corr() 확인
train_df_last['log_등록차량수'] = np.log1p(train_df_last['등록차량수'])

print("등록차량수 상관계수 : ", train_df_last.corr()['등록차량수'])
print()
print("log_등록차량수 상관계수 ; ", train_df_last.corr()['log_등록차량수'])

등록차량수 상관계수 :  총세대수          0.333440
전용면적          0.112717
전용면적별세대수      0.250513
공가수           0.118910
10분내지하철수     -0.107308
10분내버스정류장수    0.104189
단지내주차면수       0.861338
단지코드_Type     0.109893
임대건물구분_lbl   -0.449130
지역_lbl        0.060674
공급유형_lbl     -0.137277
자격유형_lbl     -0.154482
단지코드_lbl     -0.062077
qcut_총세대수     0.401309
등록차량수         1.000000
log_등록차량수     0.881679
Name: 등록차량수, dtype: float64

log_등록차량수 상관계수 ;  총세대수          0.228964
전용면적          0.111644
전용면적별세대수      0.274772
공가수           0.200793
10분내지하철수     -0.125054
10분내버스정류장수    0.080114
단지내주차면수       0.805096
단지코드_Type     0.093261
임대건물구분_lbl   -0.596565
지역_lbl        0.108532
공급유형_lbl     -0.291515
자격유형_lbl     -0.239157
단지코드_lbl     -0.065896
qcut_총세대수     0.303550
등록차량수         0.881679
log_등록차량수     1.000000
Name: log_등록차량수, dtype: float64


### 데이터 오류 처리 후, csv파일을 만들기

In [43]:
train_df.to_csv("./data/train_df.csv", index=False)
train_df_last.to_csv("./data/train_df_errno.csv", index=False)

test_df_last.to_csv("./data/test_df.csv", index=False)